# 문서 군집화 - Opinion Review dataset

데이터가 파일 단위로 들어가 있음 => 어떻게 처리하면 좋을까?

In [2]:
import pandas as pd
import glob, os

path = 'OpinosisDataset1.0/topics' # path는 폴더에 들어간 파일들을 설정 
os.path.join(path, '*.data')       # 폴더에 들어간 확장자명이 '.data'로 동일
                                   # Windows - \\, Mac - /

'OpinosisDataset1.0/topics\\*.data'

In [3]:
# 파일이름이 리스트로 나와서 반복문을 사용할 수 있다. 
# asterisk: v* => v로 시작하는 것들, *.data => .data로 끝나는 것들
all_files = glob.glob(os.path.join(path, '*.data'))
all_files[:5]

['OpinosisDataset1.0/topics\\accuracy_garmin_nuvi_255W_gps.txt.data',
 'OpinosisDataset1.0/topics\\bathroom_bestwestern_hotel_sfo.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_amazon_kindle.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_ipod_nano_8gb.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_netbook_1005ha.txt.data']

In [4]:
file = all_files[0]
file

'OpinosisDataset1.0/topics\\accuracy_garmin_nuvi_255W_gps.txt.data'

In [5]:
file.split('\\')[-1].split('.')[0]

'accuracy_garmin_nuvi_255W_gps'

In [6]:
# 리스트 컴프리헨션으로 만들 수 있다!
filename_list = []
opinion_text = []
for file in glob.glob(os.path.join(path, '*.data')):        # all_files라는 변수가 헷갈릴 수 있음!
    # 파일을 읽기 위한 3단계: open(<filename>) -> read(), write() : 작업 -> close() : 작업종료
        # open() -> read(), write()만 하고 close()를 안한다면?
        #   사용자가 혼자일 경우 문제는 없지만, 대형 서버의 경우, 서버의 사용자가 access한다면, 시스템에 있는 리소스가 계속 열려 있는 상태로 문제를 일으킬 소지가 있다
        #   이를 방지하기 위해 python에서는 open()을 with로 쓰면 with문을 벗어나는 순간 파일이 자동으로 close된다 == 리소스 낭비를 막을 수 있다.
    with open(file, encoding='latin1') as f:   # with는 keyword                                   
        text = f.read()     # with문 안에서 인덴테이션? 해야 한다.
    opinion_text.append(text)
    filename = file.split('\\')[-1].split('.')[0]
    filename_list.append(filename)

df = pd.DataFrame({'filename':filename_list, 'opinion':opinion_text})
df.head(3)

,filename,opinion
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n but for the m..."
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and ve..."
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my com...


- Simple tokenizer 함수를 이용해 feature 변환

In [7]:
from nltk import word_tokenize

def simple_tokenizer(text):             # 글자수가 2개 이하인 토큰은 제거
    # word_list = word_tokenize(text)
    # return word_list = [word for word in word_list if len(word) > 2]
    return [word for word in word_tokenize(text) if len(word) > 2]      # 간결하게 코드를 쓸 수 있다면, 가급적 간결하게 쓰는 것이 좋다 ==> pythonic한 코드를 사용하자!
    

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(tokenizer=simple_tokenizer, stop_words='english',
                        ngram_range=(1,2), min_df=0.05, max_df=0.85)
feature = tvect.fit_transform(df.opinion)       # train, test 데이터가 따로 있지 않으므로 fit_transform을 사용                         

- 군집화

In [18]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, max_iter=10000, random_state=2022)
kmeans.fit(feature)

KMeans(max_iter=10000, n_clusters=5, random_state=2022)

In [19]:
# feature = tvect.fit_transform(df.opinion) 
# 군집화(비지도학습)은 train, test가 따로 없다 
df['cluster'] = kmeans.labels_
df.head()

,filename,opinion,cluster
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n but for the m...",0
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and ve...",2
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my com...,1
3,battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\...,1
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh ...",1


In [20]:
# 편향되게 분포 되어있는 것을 확인할 수 있다. 
df.cluster.value_counts()

2    16
3    13
4    10
0     7
1     5
Name: cluster, dtype: int64

In [32]:
# 3개 집합으로 군집화
kmeans = KMeans(n_clusters=3, max_iter=10000, random_state=2022)
kmeans.fit(feature)
df['cluster_label'] = kmeans.labels_
df['cluster_label'].value_counts()

0    25
1    16
2    10
Name: cluster_label, dtype: int64

- 군집별 핵심 단어 추출하기

In [33]:
feature.shape

(51, 4154)

In [34]:
cluster_centers = kmeans.cluster_centers_
cluster_centers.shape

(3, 4154)

In [36]:
from cluster import get_cluster_details     # cluster.py에 있는 함수 이름

feature_names = tvect.get_feature_names()
cluster_details = get_cluster_details(cluster_model=kmeans, cluster_data=df,
                                      feature_names=feature_names, clusters_num=3, top_n_features=10)

In [37]:
for cluster_num, cluster_detail in cluster_details.items():
    print(f'####### Cluster {cluster_num}')
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명:', cluster_detail['filenames'][:7])
    print('==================================================')

####### Cluster 0
Top features: ['screen', 'battery', 'battery life', 'keyboard', 'kindle', 'life', 'directions', 'size', 'voice', 'speed']
Reviews 파일명: ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps']
####### Cluster 1
Top features: ['hotel', 'service', 'rooms', 'staff', 'room', 'food', 'location', 'clean', 'bathroom', 'parking']
Reviews 파일명: ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo', 'location_holiday_inn_london', 'parking_bestwestern_hotel_sfo']
####### Cluster 2
Top features: ['interior', 'mileage', 'seats', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명: ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_hond